# 🔬 ISIC Cilt Lezyonu Analizi

**Mustafa Engin Dalgıç | 254309502**

---

## ▶️ Çalıştırma:

1. **Connect** tıklayın
2. **Runtime > Run all**

---

In [ ]:
print("🚀 Başlıyor...\n")

In [ ]:
# Kütüphaneler
import os, sys, warnings, shutil
warnings.filterwarnings('ignore')

try:
    import cv2
except:
    !pip install opencv-python -q
    import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request
import zipfile

%matplotlib inline

print(f"✅ OpenCV {cv2.__version__}")
print(f"✅ NumPy {np.__version__}")
print(f"✅ Pandas {pd.__version__}\n")

In [ ]:
# Proje indirme
print("📥 Proje indiriliyor...\n")

# Temizlik
if os.path.exists("UU_DIP_ISIC_SKIN_LESION_ANALYSIS"):
    shutil.rmtree("UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
if os.path.exists("project.zip"):
    os.remove("project.zip")

# İndir
url = "https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS/archive/refs/heads/main.zip"
urllib.request.urlretrieve(url, "project.zip")
print("✅ İndirildi")

# Aç
with zipfile.ZipFile("project.zip", 'r') as z:
    z.extractall(".")
print("✅ Açıldı")

# Yeniden adlandır
try:
    os.rename("UU_DIP_ISIC_SKIN_LESION_ANALYSIS-main", "UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
    print("✅ Düzeltildi")
except:
    print("ℹ️ Klasör zaten doğru")

# Geç
os.chdir("UU_DIP_ISIC_SKIN_LESION_ANALYSIS")
print(f"✅ Konum: {os.getcwd()}\n")

In [ ]:
# ISIC kontrolü
print("🔍 ISIC klasörü...\n")

if os.path.exists("ISIC"):
    items = os.listdir("ISIC")
    print(f"✅ Bulundu ({len(items)} öğe)\n")
    for i, x in enumerate(items[:5], 1):
        print(f"   {i}. {x}")
    if len(items) > 5:
        print(f"   ...{len(items)-5} daha")
else:
    print("❌ ISIC yok")

In [ ]:
# Görüntü tarama
print("\n🔍 Görüntüler...\n")

imgs = []
for r, d, f in os.walk("ISIC"):
    for x in f:
        if x.lower().endswith(('.jpg','.jpeg','.png')):
            imgs.append(os.path.join(r, x))

print(f"✅ {len(imgs)} görüntü bulundu!\n")
if len(imgs) > 0:
    for i, x in enumerate(imgs[:3], 1):
        print(f"   {i}. {x}")

In [ ]:
# Veri yükleme
if len(imgs) == 0:
    print("❌ Görüntü yok!")
else:
    print(f"\n📊 İlk 15 görüntü yükleniyor...\n")
    
    data = []
    for p in imgs[:15]:
        try:
            im = Image.open(p)
            w, h = im.size
            data.append({
                'file': os.path.basename(p),
                'path': p,
                'width': w,
                'height': h,
                'kb': round(os.path.getsize(p)/1024, 2)
            })
        except:
            pass
    
    df = pd.DataFrame(data)
    print(f"✅ {len(df)} yüklendi!\n")
    display(df)

In [ ]:
# İstatistik
if len(imgs) > 0 and len(df) > 0:
    print(f"\n📊 İstatistik:\n")
    print(f"Çözünürlük: {df['width'].mean():.0f}x{df['height'].mean():.0f}")
    print(f"Boyut: {df['kb'].mean():.1f} KB")

In [ ]:
# Görüntü gösterimi
if len(imgs) > 0 and len(df) > 0:
    print("\n🖼️ Görüntüler...\n")
    
    fig, ax = plt.subplots(2, 2, figsize=(10, 8))
    
    for i in range(min(2, len(df))):
        im = cv2.imread(df.iloc[i]['path'])
        rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        
        # Küçült
        h, w = rgb.shape[:2]
        if max(h,w) > 400:
            s = 400/max(h,w)
            rgb = cv2.resize(rgb, None, fx=s, fy=s)
        
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
        
        ax[i,0].imshow(rgb)
        ax[i,0].set_title(f"RGB {i+1}")
        ax[i,0].axis('off')
        
        ax[i,1].imshow(gray, cmap='gray')
        ax[i,1].set_title(f"Gray {i+1}")
        ax[i,1].axis('off')
    
    plt.tight_layout()
    plt.show()
    print("✅ Tamam!")

In [ ]:
# Histogram
if len(imgs) > 0 and len(df) > 0:
    print("\n📊 Histogram...\n")
    
    im = cv2.imread(df.iloc[0]['path'])
    rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    h, w = rgb.shape[:2]
    if max(h,w) > 400:
        s = 400/max(h,w)
        rgb = cv2.resize(rgb, None, fx=s, fy=s)
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    
    hr = cv2.calcHist([rgb],[0],None,[256],[0,256])
    hg = cv2.calcHist([rgb],[1],None,[256],[0,256])
    hb = cv2.calcHist([rgb],[2],None,[256],[0,256])
    hgray = cv2.calcHist([gray],[0],None,[256],[0,256])
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    
    ax[0].plot(hr, 'r', alpha=0.7)
    ax[0].plot(hg, 'g', alpha=0.7)
    ax[0].plot(hb, 'b', alpha=0.7)
    ax[0].set_title('RGB')
    
    ax[1].plot(hgray, 'k')
    ax[1].set_title('Grayscale')
    
    plt.tight_layout()
    plt.show()
    print("✅ Tamam!")

In [ ]:
# CLAHE
if len(imgs) > 0 and len(df) > 0:
    print("\n✨ CLAHE...\n")
    
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(gray)
    
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    
    ax[0].imshow(gray, cmap='gray')
    ax[0].set_title('Orijinal')
    ax[0].axis('off')
    
    ax[1].imshow(enhanced, cmap='gray')
    ax[1].set_title('CLAHE')
    ax[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    print("✅ Tamam!")

---

## 🎉 BAŞARILI!

### Tam program:

In [ ]:
if len(imgs) > 0:
    print("🚀 Tam program (10-15 dk)...\n")
    !python isic_project.py

---

**Mustafa Engin Dalgıç**  
engindalgic86@gmail.com  
[GitHub](https://github.com/engindalgic86-cpu/UU_DIP_ISIC_SKIN_LESION_ANALYSIS)